# Импорт библиотек

In [1]:
from bs4 import BeautifulSoup
import requests
import json
import time
from pprint import pprint
import re


# Парсинг ссылок на посты

In [2]:
items_href = []

In [3]:
for i in range(1,41):
    url = 'https://www.djimsk.ru/catalog/products/?PAGEN_1='+str(i)

    headers = {
        'Accept': '*/*',
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.167 YaBrowser/22.7.3.799 Yowser/2.5 Safari/537.36'
    }

    req = requests.get(url, headers=headers)
    src = req.text

    soup = BeautifulSoup(src, 'lxml')

    allHrefs = soup.find_all(class_="productColText")



    for item in allHrefs:
        print(item)
        print(' ')
        itemText = item.text
        find_a = item.find("a")
        itemHref = find_a.get('href')
        print(itemText)
        print(itemHref)
        print(' ')
        print(' ')
        items_href.append(
            {
                "title": itemText,
                "url": itemHref
            }
        )


    with open('djimsik_items.json', 'w') as file:
        json.dump(items_href, file, indent=4, ensure_ascii=False)


<div class="productColText">
<a class="name" data-product-id="42415" href="/catalog/products/ronin/dji_rs_3/stabilizator_dji_rs_3_pro_combo.html">Стабилизатор DJI RS 3 Pro Combo</a>
</div>
 

Стабилизатор DJI RS 3 Pro Combo

/catalog/products/ronin/dji_rs_3/stabilizator_dji_rs_3_pro_combo.html
 
 
<div class="productColText">
<a class="name" data-product-id="42414" href="/catalog/products/ronin/dji_rs_3/stabilizator_dji_rs_3_combo.html">Стабилизатор DJI RS 3 Combo</a>
</div>
 

Стабилизатор DJI RS 3 Combo

/catalog/products/ronin/dji_rs_3/stabilizator_dji_rs_3_combo.html
 
 
<div class="productColText">
<a class="name" data-product-id="42410" href="/catalog/products/ronin/dji_rs_3/stabilizator_dji_rs_3_pro.html">Стабилизатор DJI RS 3 Pro</a>
</div>
 

Стабилизатор DJI RS 3 Pro

/catalog/products/ronin/dji_rs_3/stabilizator_dji_rs_3_pro.html
 
 
<div class="productColText">
<a class="name" data-product-id="42408" href="/catalog/products/ronin/dji_rs_3/stabilizator_dji_rs_3.html">Стабили

# Сбор всевозможных данных по товару

### Загрузка URLs в переменную data

In [4]:
with open('djimsik_items.json') as file:
    data = json.load(file)
    

## Проверка на первом URL

In [76]:
url = 'https://www.djimsk.ru'+data[4]['url']
url

'https://www.djimsk.ru/catalog/products/mavic/dji_mini_3_pro/kvadrokopter_dji_mini_3_pro.html'

### Здесь будут храниться словари с данными

In [57]:
dictionary_repository = []

### Тест

In [66]:
#url = 'https://www.djimsk.ru'+data[0]['url']

headers = {
    'Accept': '*/*',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.167 YaBrowser/22.7.3.799 Yowser/2.5 Safari/537.36'
}

req = requests.get(url, headers=headers)
time.sleep(1)
src = req.text

soup = BeautifulSoup(src, 'lxml')

name = soup.find(class_="changeName")
price = soup.find(class_="price changePrice noPrice")
all_data = soup.find_all("td")

info_name = []
info_descr = []

for i in range(len(all_data)):
    if i%2==0:
        x = all_data[i].text
        x = re.sub('\s+', ' ', x)
        info_name.append(x)       
    else:
        z = all_data[i].text
        z = re.sub('\s+', ' ', z)
        info_descr.append(z)

store = dict(zip(info_name, info_descr))


name_and_price = dict({
    "name": name.text,
    "price": price.text
})

name_and_price_and_store = name_and_price | store

dictionary_repository.append(name_and_price_and_store)



# with open('djimsik_items_data.json', 'w') as file:
#     json.dump(output, file, indent=4, ensure_ascii=False)


{' FOV ': ' 57,4° (по горизонтали), 44,6° (по вертикали), 70,1° (по '
          'диагонали) ',
 ' Беспроводная связь ': ' Bluetooth 5.0 ',
 ' Вес ': ' Стабилизатор: около 1143 г | Рукоятка: около 265 г | Удлинитель '
          'рукоятки / штатив (металлический): около 226 г | Верхняя и нижняя '
          'быстросъемная площадки: около 107 г ',
 ' Время зарядки ': ' около 1,5 часов (при использовании зарядного устройства '
                    'на 24 Вт, рекомендуется использовать зарядные устройства '
                    'с протоколом QC 2.0 или PD ',
 ' Время работы ': ' 12 часов ',
 ' Диапазон обнаружения ': ' от 0,5 до 14 м ',
 ' Диапазон рабочих частот ': ' 2,400–2,484 ГГц ',
 ' Емкость ': ' 1950 мАч ',
 ' Макс. дальность передачи сигнала ': ' 200 м (SRRC/FCC) | 100 м (CE) ',
 ' Макс. контролируемая угловая скорость ': ' Наклон: 360 °/с; Крен: 360 °/с; '
                                            'Поворот: 360 °/с ',
 ' Механический диапазон углов вращения ': ' Ось поворота: непрер

# Сбор со всех ссылок

In [5]:
dictionary_repository = []
counter = 0
for i in data:
    try:
    
        url = 'https://www.djimsk.ru'+str(i['url'])
        print(url)

        headers = {
            'Accept': '*/*',
            'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.5005.167 YaBrowser/22.7.3.799 Yowser/2.5 Safari/537.36'
        }

        req = requests.get(url, headers=headers)
        time.sleep(1)
        src = req.text

        soup = BeautifulSoup(src, 'lxml')

        name = soup.find(class_="changeName")
        price_html = soup.find(class_="flex-row")
        all_data = soup.find_all("td")

        info_name = []
        info_descr = []

        for i in range(len(all_data)):
            if i%2==0:
                x = all_data[i].text
                x = re.sub('\s+', ' ', x)
                info_name.append(x)       
            else:
                z = all_data[i].text
                z = re.sub('\s+', ' ', z)
                info_descr.append(z)

        store = dict(zip(info_name, info_descr))

        price = price_html.text
        price = re.sub('\s+', ' ', price)

        name_price_specific = dict({
            "name": name.text,
            "price": price,
            "specifications": store
        })

        dictionary_repository.append(name_price_specific)
        
        
        pprint(name_price_specific)
        print(url + ' ready', counter)
        counter += 1

    except Exception as _ex:
        print(_ex)
        counter += 1
    
with open('djimsik_items_data.json', 'w') as file:
    json.dump(dictionary_repository, file, indent=4, ensure_ascii=False)





https://www.djimsk.ru/catalog/products/ronin/dji_rs_3/stabilizator_dji_rs_3_pro_combo.html
{'name': 'Стабилизатор DJI RS 3 Pro Combo',
 'price': ' Цена по запросу ',
 'specifications': {' FOV ': ' 57,4° (по горизонтали), 44,6° (по вертикали), '
                             '70,1° (по диагонали) ',
                    ' Беспроводная связь ': ' Bluetooth 5.0 ',
                    ' Вес ': ' Стабилизатор: около 1143 г | Рукоятка: около '
                             '265 г | Удлинитель рукоятки / штатив '
                             '(металлический): около 226 г | Верхняя и нижняя '
                             'быстросъемная площадки: около 107 г ',
                    ' Время зарядки ': ' около 1,5 часов (при использовании '
                                       'зарядного устройства на 24 Вт, '
                                       'рекомендуется использовать зарядные '
                                       'устройства с протоколом QC 2.0 или PD ',
                    ' Время рабо